<a href="https://colab.research.google.com/github/Ecevangelista/422_RNN-Stock-Price/blob/main/Assign9_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount = True)

# Ingest

In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import statistics
from os import listdir
import yfinance as yf
from shutil import copyfile
import numpy as np
from numpy import arange, asarray, save, load
import statsmodels
import statsmodels.api as sm
import math
from matplotlib.image import imread
from random import seed
from random import random
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Conv1D
from keras.layers import MaxPooling2D
from keras.layers import Dense, LSTM, SimpleRNN
from keras.layers import Flatten, Dropout, TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.metrics import MeanSquaredError

In [ ]:
#download s&p500 index data for 5 years, 2015 - 2019

gspc = yf.Ticker("^GSPC")
#print(gspc)

datagspc = gspc.history(start = "2015-01-01", end="2019-12-31")

# EDA

In [ ]:
datagspc.sample(10)

In [ ]:
datagspc.describe()

In [ ]:
type(datagspc)

In [ ]:
dataclose = datagspc[['Close']]

Looking the Closing prices to look at trends over the 5 years:
There is a slight dip between 2015 - 2016 and in early 2019. Closing prices 
have seen an increase over the past 5 years otherwise. The data will be split into allocating the 1st 3 years (2015-2017) to the Train set, 2018 to Validation, and 2019 to Test. 2018 has a high degree of volatility vs. the earlier years, so this will affect model fitting.  

In [ ]:
#plot dataclose

plt.plot(dataclose)
plt.title("Closing Prices S&P Index 2015 - 2019")
plt.show


# Preprocessing

In [ ]:
#convert pandas df to values to prepare for keras

datasetf = dataclose.values
datasetf = datasetf.astype('float32')

In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
datasetf = scaler.fit_transform(datasetf)

In [ ]:

# split into samples
X_samples = list()
y_samples = list()
 
NumerOfRows = len(datasetf)
TimeSteps=5  # next day's Price Prediction is based on past 5 days' prices
 
# Iterate thru the values to create combinations
for i in range(TimeSteps , NumerOfRows , 1):
    x_sample = datasetf[i-TimeSteps:i]
    y_sample = datasetf[i]
    X_samples.append(x_sample)
    y_samples.append(y_sample)
 
################################################
# Reshape the Input as a 3D (number of samples, Time Steps, Features)
X_data=np.array(X_samples)
X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 1)
print('\n#### Input Data shape ####')
print(X_data.shape)
 
# We do not reshape y as a 3D data  as it is supposed to be a single column only
y_data=np.array(y_samples)
y_data=y_data.reshape(y_data.shape[0], 1)
print('\n#### Output Data shape ####')
print(y_data.shape)

The dataset is split into 60% Train, 20% Validation, 20% Test.

In [ ]:
# Splitting the data into train and test
X_train=X_data[:752]
X_valid = X_data[752:1002]
X_test=X_data[1002:]
y_train=y_data[:752]
y_valid = y_data[752:1002]
y_test=y_data[1002:]
 
############################################
 
# Printing the shape of training and testing
print('\n#### Training Data shape ####')
print(X_train.shape)
print(y_train.shape)
print('\n#### Valid Data shape ####')
print(X_valid.shape)
print(y_valid.shape)
print('\n#### Testing Data shape ####')
print(X_test.shape)
print(y_test.shape)

# Baseline Model - Keras Linear regression

In [ ]:
base_model = Sequential([
    keras.layers.Flatten(input_shape=[5,1]),
    keras.layers.Dense(1)                                      
])

base_model.compile(optimizer='adam', loss='MeanSquaredError')
history = base_model.fit(X_train, y_train, epochs = 10, batch_size = 1, verbose =2)

In [ ]:
# make predictions
trainPredict = base_model.predict(X_train)
validPredict = base_model.predict(X_valid)
testPredict = base_model.predict(X_test)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainYbase = scaler.inverse_transform(y_train)

validPredict = scaler.inverse_transform(validPredict)
validYbase = scaler.inverse_transform(y_valid)

testPredict = scaler.inverse_transform(testPredict)
testYbase = scaler.inverse_transform(y_test)

In [ ]:
#BASELINE RMSE

trainrmsebase = mean_squared_error(trainYbase, trainPredict[:,0], squared = False)
validrmsebase = mean_squared_error(validYbase, validPredict[:,0], squared = False)
testrmsebase = mean_squared_error(testYbase, testPredict[:,0], squared = False)

print("Train RMSE Base", trainrmsebase)
print("Valid RMSE Base", validrmsebase)
print("Test RMSE Base", testrmsebase)


# Deep RNN Model

In [ ]:
deepmodel = Sequential([
    keras.layers.SimpleRNN(20, return_sequences = True, input_shape = [None,1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)                                      
])

deepmodel.compile(optimizer='adam', loss='MeanSquaredError')
history = deepmodel.fit(X_train, y_train, epochs = 10, batch_size = 1, verbose =2)

In [ ]:
# make predictions
trainPredictd = deepmodel.predict(X_train)
validPredictd = deepmodel.predict(X_valid)
testPredictd = deepmodel.predict(X_test)
# invert predictions
trainPredictd = scaler.inverse_transform(trainPredictd)
trainYd = scaler.inverse_transform(y_train)
validPredictd = scaler.inverse_transform(validPredictd)
validYd = scaler.inverse_transform(y_valid)
testPredictd = scaler.inverse_transform(testPredictd)
testYd = scaler.inverse_transform(y_test)

In [ ]:
#RMSE

trainrmsed = mean_squared_error(trainYd, trainPredictd[:,0], squared = False)
validrmsed = mean_squared_error(validYd, validPredictd[:,0], squared = False)
testrmsed = mean_squared_error(testYd, testPredictd[:,0], squared = False)

print("Train RMSE Deep", trainrmsed)
print("Valid RMSE Deep", validrmsed)
print("Test RMSE Deep", testrmsed)


In [ ]:
#Graph TEST predicted vs actual DEEP RNN

plt.plot(testPredictd, color = 'green', label = 'Predicted Close')
plt.plot(testYd, color = 'blue', label = 'Original Close')
 
plt.title('Deep RNN Stock Price Predictions on Test')
plt.ylabel('Stock Price')
 
plt.legend()
fig=plt.gcf()
fig.set_figwidth(15)
fig.set_figheight(7)
plt.show()

# LSTM Model 1

In [ ]:
model1 = Sequential([
    keras.layers.LSTM(20, return_sequences = True, input_shape = [None,1]),
    keras.layers.LSTM(20),
    keras.layers.Dense(1)                                      
])

model1.compile(optimizer='adam', loss='MeanSquaredError')
history = model1.fit(X_train, y_train, epochs = 10, batch_size = 1, verbose =2)

In [ ]:
# make predictions
trainPredictmod1 = model1.predict(X_train)
validPredictmod1 = model1.predict(X_valid)
testPredictmod1 = model1.predict(X_test)
# invert predictions
trainPredictmod1 = scaler.inverse_transform(trainPredictmod1)
trainYmod1 = scaler.inverse_transform(y_train)
validPredictmod1 = scaler.inverse_transform(validPredictmod1)
validYmod1 = scaler.inverse_transform(y_valid)
testPredictmod1 = scaler.inverse_transform(testPredictmod1)
testYmod1 = scaler.inverse_transform(y_test)

In [ ]:
## RMSE LSTM MODEL 1

trainrmsemod1 = mean_squared_error(trainYmod1, trainPredictmod1, squared = False)
validrmsemod1 = mean_squared_error(validYmod1, validPredictmod1, squared = False)
testrmsemod1 = mean_squared_error(testYmod1, testPredictmod1, squared = False)

print("Train RMSE LSTM", trainrmsemod1)
print("Valid RMSE LSTM", validrmsemod1)
print("Test RMSE LSTM", testrmsemod1)

In [ ]:
#Graph TEST predicted vs actual LSTM Model 1

plt.plot(testPredictmod1, color = 'green', label = 'Predicted Close')
plt.plot(testYmod1, color = 'blue', label = 'Original Close')
 
plt.title('LSTM Model 1 Stock Price Predictions on Test')
plt.ylabel('Stock Price')
 
plt.legend()
fig=plt.gcf()
fig.set_figwidth(15)
fig.set_figheight(7)
plt.show()

# LSTM Model 2: 
- Tuned with batch size and length of epochs
- Training included monitoring validation loss

In my original model, I used ModelCheckpoint due to the larger number of epochs and have commented it out below. Instead of activating ModelCheckpoint, I will use the best model that was previously saved to calculate the performance.

In [ ]:
model5 = Sequential([
    keras.layers.LSTM(20, return_sequences = True, input_shape = [None,1]),
    keras.layers.LSTM(20),
    keras.layers.Dense(1)                                      
])

model5.compile(optimizer='adam', loss='MeanSquaredError', metrics = [keras.metrics.MeanSquaredError()])

#creating callback
#filepath = '/content/drive/MyDrive/422NU/assign9/model5.epoch{epoch:03d}.h5'

#checkpoint = ModelCheckpoint(filepath = filepath,
#                             monitor='val_loss', verbose=1,
#                             save_best_only = True,
#                             mode = 'min')


history = model5.fit(X_train, y_train, epochs = 50, batch_size = 4, verbose = 2, validation_data=(X_valid, y_valid))

In [ ]:
model5ld = load_model('/content/drive/MyDrive/422NU/assign9/model5.epoch047.h5')

# make predictions LSTM4
trainPredictmod5 = model5ld.predict(X_train)
validPredictmod5 = model5ld.predict(X_valid)
testPredictmod5 = model5ld.predict(X_test)
# invert predictions
trainPredictmod5 = scaler.inverse_transform(trainPredictmod5)
trainYmod5 = scaler.inverse_transform(y_train)
validPredictmod5 = scaler.inverse_transform(validPredictmod5)
validYmod5 = scaler.inverse_transform(y_valid)
testPredictmod5 = scaler.inverse_transform(testPredictmod5)
testYmod5 = scaler.inverse_transform(y_test)

In [ ]:
## RMSE LSTM MODEL 2

trainrmsemod5 = mean_squared_error(trainYmod5, trainPredictmod5, squared = False)
validrmsemod5 = mean_squared_error(validYmod5, validPredictmod5, squared = False)
testrmsemod5 = mean_squared_error(testYmod5, testPredictmod5, squared = False)

print("Train RMSE LSTM 5", trainrmsemod5)
print("Valid RMSE LSTM 5", validrmsemod5)
print("Test RMSE LSTM 5", testrmsemod5)

In [ ]:
#Graph TEST predicted vs actual LSTM Model 2

plt.plot(testPredictmod5, color = 'green', label = 'Predicted Close')
plt.plot(testYmod5, color = 'blue', label = 'Original Close')
 
plt.title('LSTM Model 2 Stock Price Predictions on Test')
plt.ylabel('Stock Price')
 
plt.legend()
fig=plt.gcf()
fig.set_figwidth(15)
fig.set_figheight(7)
plt.show()

# Conclusion

The Deep RNN Model performed best with 10 epochs achieving an RMSE of 23.65 on the Test set. 
LSTM Model 2 performed slightly worse, achieving an RMSE of 26.46 on the Test set. Including the validation data during training helped to mitigate overfitting, since the RMSE of the Validation set was higher than that of the Test set due to the price volatility at that time.